In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms

import numpy as numpy
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

from networks.LeNet import LeNet
from GTSRB_Reader import GTSRB_Reader
from datasets.GTSRB_Dataset import GTSRB_Dataset
from adversarial_examples.Adversary_Dataset import Adversary_Dataset
from adversarial_examples.Mixed_Dataset import Mixed_Dataset

In [18]:
# epsilons : 변형 정도
epsilons = [.05, .1]#[.001, .015, .02, .025, .03, .04] # epsilon 0.05도 굉장히 티가 많이 나기 때문에 티가 덜 나는 쪽으로 하기 위해서 추가
pretrained_model = './models/LeNet/lenet_GTSRB_net.pth'
use_cuda = True

NUM_CLASSES = 43

In [19]:
# dataset = datasets.CIFAR10(
#     root='./datasets/cifar10/',
#     train=True,
#     transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
#     ])
# )
""" train 
g = GTSRB_Reader()

clean_train_data_examples, _ = g.readTrafficSigns('./datasets/GTSRB/training/Images')

tf = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

dataset = Adversary_Dataset(clean_train_data_examples, tf)
data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1)

clean_train_data_examples = []
clean_train_label_examples = []
for (data, label) in tqdm(data_loader):
    clean_train_data_examples.append(data[0])
    clean_train_label_examples.append(label[0])

clean_train_dataset = Mixed_Dataset(clean_train_data_examples, clean_train_label_examples)
data_loader = torch.utils.data.DataLoader(dataset=clean_train_dataset, batch_size=1, shuffle=True)
"""
# """ test
g = GTSRB_Reader()

clean_test_data_examples, clean_test_label_examples = g.readTestTrafficSigns('./datasets/GTSRB/test/Images/')

tf = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

dataset = GTSRB_Dataset(clean_test_data_examples, clean_test_label_examples, tf)
data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1, shuffle=True)
# """
# dataset = Mixed_Dataset(clean_train_data_examples, clean_train_label_examples)

# data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1, shuffle=False)

print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

model = LeNet(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))
model.eval()

CUDA Available:  True


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=43, bias=True)
)

### FGSM attacks

In [20]:
# FGSM 공격 코드
def fgsm_attack(image, epsilon, data_grad):

    sign_data_grad = data_grad.sign()

    perturbed_image = image + epsilon * sign_data_grad
    # perturbed_image = torch.clamp(perturbed_image, 0, 1) # for grayscale
    perturbed_image = torch.clamp(perturbed_image, 0, 255)

    return perturbed_image

### Test function

In [21]:
def test( model, device, data_loader, epsilon ):
    # correct = 0
    adv_examples = []

    for data, target in tqdm(data_loader):
        data, target = data.to(device), target.to(device)

        data.requires_grad = True

        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]

        loss = F.cross_entropy(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)
        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]
        # if final_pred.item() == target.item():
        #     correct += 1
            # adv_ex = perturbed_data.cpu()
            # adv_examples.append( (target.item(), final_pred.item(), adv_ex) )
        
        adv_ex = perturbed_data.cpu()
        adv_examples.append( (target.item(), adv_ex) )
        
    # final_acc = correct / 50_000
    # print("Epsilon: {}\tTest Accuracy = {}".format(epsilon, final_acc))

    return adv_examples
    # 정확도와 적대적 예제를 리턴합니다
    # return final_acc, adv_examples

In [25]:
accs = []
adv_examples = []

ad_ex = test(model, device, data_loader, epsilons[1])
adv_examples.append(ad_ex)

# for eps in epsilons:
#     #acc, ex = test(model, device, test_loader, eps)
#     ad_ex = test(model, device, test_loader, eps)
#     # accs.append(acc)
#     adv_examples.append(ad_ex) 

100%|██████████| 12630/12630 [00:55<00:00, 228.89it/s]


### Save adversarial examples

In [26]:
import os

PATH = './adversarial_examples/'
DATASET = 'GTSRB/' # CIFAR10, CIFAR100
TYPE = 'test/' # training, test
EPSILON = str(epsilons[1])

# 각 엡실론에서 적대적 샘플 저장
tf = transforms.ToPILImage()

# for eps in epsilons:
if not os.path.isdir(PATH + DATASET + TYPE + EPSILON):
    os.mkdir(PATH + DATASET + TYPE + EPSILON)

for i in range(0, NUM_CLASSES):
    if not os.path.isdir(PATH + DATASET + TYPE + EPSILON + '/' + str(i)):
        os.mkdir(PATH + DATASET + TYPE + EPSILON + '/' + str(i))

# for i in range(len(epsilons)):
cnt = [0] * NUM_CLASSES
for j in tqdm(range(len(adv_examples[0]))):
    orig, ex = adv_examples[0][j]
    cnt[orig] += 1

    tf(ex[0]).save(PATH + DATASET + TYPE + EPSILON  + '/' + str(orig) + '/' + str(cnt[orig]) + '.jpg')

100%|██████████| 12630/12630 [00:08<00:00, 1450.21it/s]
